In [10]:
import sys
sys.path.insert(0, r'../../libraries/quetzal')
sys.path.insert(0, r'../../libraries/syspy')

from quetzal.io import  gtfs_importer
from quetzal.model import stepmodel, model

import geopandas as gpd

In [11]:
training_folder = '../../'
input_folder = training_folder + r'gis/inputs/'

In [12]:
sm = stepmodel.read_zip(training_folder + 'model/preparation/base.zip')

read_hdf: : 100%|██████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 69.18it/s]


# add road

In [13]:
road_links = gpd.read_file(input_folder+'road_links.geojson', driver='GeoJSON')
road_nodes = gpd.read_file(input_folder+'road_nodes.geojson', driver='GeoJSON')
road_nodes.set_index('osmid', inplace=True)

sm.road_links = road_links
sm.road_nodes = road_nodes

In [14]:
sm = sm.change_epsg(2154, 'meter')

Reprojecting model from epsg 4326 to epsg 2154: 100%|██████████████████████████████████| 28/28 [00:01<00:00, 17.81it/s]


# road integrity

In [15]:
sm.integrity_fix_collision()

can not add prefixes on table:  footpaths


## road_fix

In [16]:
sm.integrity_fix_road_network(cutoff=10, recursive_depth=5)

direct: 10147it [00:04, 2492.52it/s]
reversed: 10147it [00:04, 2533.61it/s]
9989it [00:03, 2541.11it/s]
direct: 9989it [00:04, 2392.05it/s]
reversed: 9989it [00:04, 2477.46it/s]
9987it [00:04, 2480.04it/s]


## road_test

In [17]:
sm.integrity_test_dead_ends()

9987it [00:00, 11059.95it/s]


In [18]:
sm.integrity_fix_nodeset_consistency()

In [19]:
sm.integrity_test_isolated_roads()

# Transport Integrity

## transport_fix

In [20]:
sm.integrity_fix_collision()
sm.integrity_fix_circular_lines()
sm.integrity_fix_sequences()

dropped broken sequences: ['1026572651003316', '126356011531123', '7826999790881055', '7126352601082481', '6026797241770434', '1026353381829392', '1026572650941049', '1026618981581054', '7026391111663108', '11526352000884475', '1026618641034640', '1026999531904138', '6026797241770433', '1026353381829393', '8226999791364557']


## transport_test

In [21]:
sm.integrity_test_circular_lines()
sm.integrity_test_collision()
sm.integrity_test_nodeset_consistency()
sm.integrity_test_sequences()

In [22]:
sm.integrity_test_all(errors='raise')

passed: integrity_test_circular_lines
passed: integrity_test_collision


9987it [00:00, 11146.42it/s]


passed: integrity_test_dead_ends
passed: integrity_test_isolated_roads
passed: integrity_test_nodeset_consistency


9987it [00:03, 2555.56it/s]


passed: integrity_test_road_network
passed: integrity_test_sequences


# to_file

In [24]:
sm.to_zip(training_folder + 'model/preparation/road.zip')   

to_hdf(new file): 100%|████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 15.02it/s]


could not save attribute: missing_nodes
could not save attribute: missing_road_nodes
